<a href="https://colab.research.google.com/github/wannasmile/colab_code_note/blob/main/QUANT025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 导入必要的库
import math
from math import floor
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

# 下载数据
def download_stock_data():
    # 定义需要处理的股票代码列表
    stocks = ['000001.SS', 'AAPL', 'BTC-USD', 'DJI', 'GSPC', 'IXIC']

    for stock in stocks:
        try:
            filename = f'https://raw.githubusercontent.com/wannasmile/colab_code_note/refs/heads/main/{stock}.csv'
            df = pd.read_csv(filename,
                           delimiter=',',
                           usecols=['Date','Open','High','Low','Close','Adj Close','Volume'])
            df = df.sort_values('Date')
            print(f"{'#' * 50}\n{stock}")
            print(df.head(2))
            print('#' * 50)
            df.to_csv(f'{stock}.csv', index=False)
        except Exception as e:
            print(f"Error downloading {stock}: {str(e)}")


lookahead = 2

class Config:
    def __init__(self):
        # 序列参数
        self.batch_size = 64
        self.src_len = 30     # 输入序列长度
        self.dec_len = lookahead      # 解码器初始序列长度
        self.tgt_len = lookahead      # 预测步数

        # 特征维度
        self.num_features = 5  # 输入特征数(Open, High, Low, Adj Close, Volume)
        self.output_size = 1   # 输出维度(预测Adj Close)

        # 模型参数
        self.d_model = 512
        self.dense_dim = 2048
        self.dim_feedforward = 2048
        self.num_heads = 8
        self.num_encoder_layers = 6
        self.num_decoder_layers = 6
        self.dropout_rate = 0.1

        # 训练参数
        self.epochs = 200
        self.learning_rate = 0.01
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.teacher_forcing_ratio = 0.5

        # 验证序列长度设置的合理性
        assert self.tgt_len <= self.dec_len, "预测步数不能大于解码器输入长度"
        assert self.dec_len <= self.src_len, "解码器输入长度不能大于编码器输入长度"
        assert self.output_size <= self.num_features, "输出维度不能大于输入特征数"

class Galformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        # 输入投影层
        self.input_projection = nn.Sequential(
            nn.Linear(config.num_features, config.d_model),
            nn.ReLU(),
            nn.LayerNorm(config.d_model)
        )

        # 编码器
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=config.d_model,
            nhead=config.num_heads,
            dim_feedforward=config.dim_feedforward,
            dropout=config.dropout_rate,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=config.num_encoder_layers
        )

        # 解码器
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=config.d_model,
            nhead=config.num_heads,
            dim_feedforward=config.dim_feedforward,
            dropout=config.dropout_rate,
            batch_first=True
        )
        self.decoder = nn.TransformerDecoder(
            decoder_layer,
            num_layers=config.num_decoder_layers
        )

        # 输出投影层
        self.output_projection = nn.Sequential(
            nn.Linear(config.d_model, config.dense_dim),
            nn.ReLU(),
            nn.LayerNorm(config.dense_dim),
            nn.Linear(config.dense_dim, config.output_size)
        )

        # 位置编码
        self.register_buffer('pos_encoding', self._create_pos_encoding(config))

        # 生成前瞻掩码
        self.register_buffer(
            'causal_mask',
            self._generate_square_subsequent_mask(config.tgt_len)
        )

    def _create_pos_encoding(self, config):
        """创建位置编码"""
        pe = torch.zeros(config.src_len, config.d_model)
        position = torch.arange(0, config.src_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, config.d_model, 2).float() *
                           (-math.log(10000.0) / config.d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe.unsqueeze(0)

    def _generate_square_subsequent_mask(self, sz):
        """生成前瞻掩码"""
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf'))
        mask = mask.masked_fill(mask == 1, float(0.0))
        return mask




    def forward(self, src, target=None, teacher_forcing_ratio=0.5):
        """
        src: (batch_size, src_len, num_features)
        target: (batch_size, tgt_len, output_size) 用于teacher forcing
        Returns: (batch_size, tgt_len, output_size)
        """
        batch_size = src.size(0)

        # 1. 输入投影
        src = self.input_projection(src)  # (batch_size, src_len, d_model)

        # 2. 添加位置编码
        src = src + self.pos_encoding[:, :src.size(1), :]

        # 3. 编码器处理
        memory = self.encoder(src)  # (batch_size, src_len, d_model)

        # 4. 准备解码器输入
        dec_input = src[:, -self.config.dec_len:, :]  # (batch_size, dec_len, d_model)

        # 5. 多步预测
        outputs = []
        for t in range(self.config.tgt_len):
            # 解码器处理
            dec_output = self.decoder(
                dec_input,
                memory,
                tgt_mask=self.causal_mask[:dec_input.size(1), :dec_input.size(1)]
            )  # (batch_size, dec_len, d_model)

            # 输出投影
            step_output = self.output_projection(dec_output[:, -1:, :])  # (batch_size, 1, output_size)
            outputs.append(step_output)

            # 更新解码器输入
            if (teacher_forcing_ratio > random.random()) and self.training and target is not None:
                # 使用目标值作为下一步输入，需要投影到d_model维度
                next_input = target[:, t:t+1, :]  # (batch_size, 1, output_size)
                next_input = self.input_projection(
                    F.pad(next_input, (0, self.config.num_features - self.config.output_size))
                )  # (batch_size, 1, d_model)
            else:
                # 使用预测值作为下一步输入，需要投影到d_model维度
                next_input = self.input_projection(
                    F.pad(step_output, (0, self.config.num_features - self.config.output_size))
                )  # (batch_size, 1, d_model)

            # 更新解码器输入
            dec_input = torch.cat([dec_input, next_input], dim=1)  # (batch_size, dec_len+1, d_model)
            # 如果序列太长，截断最早的时间步
            if dec_input.size(1) > self.config.dec_len:
                dec_input = dec_input[:, 1:, :]

        # 合并所有时间步的输出
        outputs = torch.cat(outputs, dim=1)  # (batch_size, tgt_len, output_size)
        return outputs


def custom_loss(y_pred, y_true):
    """混合损失函数：MSE + 趋势准确率惩罚"""
    # MSE损失
    mse_loss = F.mse_loss(y_pred, y_true)

    # 趋势准确率
    pred_diff = y_pred[:, 1:] - y_pred[:, :-1]
    true_diff = y_true[:, 1:] - y_true[:, :-1]

    pred_trend = (pred_diff > 0).float()
    true_trend = (true_diff > 0).float()

    trend_accuracy = torch.mean((pred_trend == true_trend).float())

    # 动态缩放因子
    scale_factor = 10 ** math.floor(math.log10(mse_loss.item()))

    # 总损失
    total_loss = mse_loss + (1 - trend_accuracy) * scale_factor

    return total_loss, trend_accuracy



def prepare_data(df, config, test_size=0.1, valid_size=0.1):
    """准备时序预测数据，使用残差（差分）预测"""
    # 选择特征
    features = ['Open', 'High', 'Low', 'Adj Close', 'Volume']
    target = 'Adj Close'

    # 1. 计算目标变量的差分
    df['Adj Close_diff'] = df['Adj Close'].diff()
    df = df.dropna()  # 删除第一行（差分后的NaN）

    # 2. 划分数据集
    train_size = int(len(df) * (1 - test_size - valid_size))
    valid_size = int(len(df) * valid_size)

    train_df = df[:train_size]
    valid_df = df[train_size:train_size + valid_size]
    test_df = df[train_size + valid_size:]

    print(f"数据集大小: 训练集 {len(train_df)}, 验证集 {len(valid_df)}, 测试集 {len(test_df)}")

    # 3. 标准化
    scalers = {}
    train_scaled = pd.DataFrame()
    valid_scaled = pd.DataFrame()
    test_scaled = pd.DataFrame()

    # 对原始特征进行标准化
    for column in features:
        if column == 'Volume':
            scalers[column] = StandardScaler()
        else:
            scalers[column] = MinMaxScaler()

        train_scaled[column] = scalers[column].fit_transform(train_df[column].values.reshape(-1, 1)).flatten()
        valid_scaled[column] = scalers[column].transform(valid_df[column].values.reshape(-1, 1)).flatten()
        test_scaled[column] = scalers[column].transform(test_df[column].values.reshape(-1, 1)).flatten()

    # 对差分序列单独进行标准化
    scalers['diff'] = StandardScaler()
    train_scaled['Adj Close_diff'] = scalers['diff'].fit_transform(train_df['Adj Close_diff'].values.reshape(-1, 1)).flatten()
    valid_scaled['Adj Close_diff'] = scalers['diff'].transform(valid_df['Adj Close_diff'].values.reshape(-1, 1)).flatten()
    test_scaled['Adj Close_diff'] = scalers['diff'].transform(test_df['Adj Close_diff'].values.reshape(-1, 1)).flatten()

    def create_sequences(data, seq_len, pred_len):
        """使用滑动窗口创建序列"""
        X, y = [], []
        for i in range(len(data) - seq_len - pred_len + 1):
            # 输入序列（包含所有特征）
            X.append(data[features].iloc[i:i + seq_len].values)
            # 目标序列（使用差分值）
            y.append(data['Adj Close_diff'].iloc[i + seq_len:i + seq_len + pred_len].values)
        return np.array(X), np.array(y).reshape(-1, pred_len, 1)

    # 4. 创建序列
    X_train, y_train = create_sequences(train_scaled, config.src_len, config.tgt_len)
    X_valid, y_valid = create_sequences(valid_scaled, config.src_len, config.tgt_len)
    X_test, y_test = create_sequences(test_scaled, config.src_len, config.tgt_len)

    print("\n序列形状:")
    print(f"训练集 - 输入: {X_train.shape}, 目标: {y_train.shape}")
    print(f"验证集 - 输入: {X_valid.shape}, 目标: {y_valid.shape}")
    print(f"测试集 - 输入: {X_test.shape}, 目标: {y_test.shape}")

    # 5. 创建数据加载器
    train_dataset = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    valid_dataset = TensorDataset(torch.FloatTensor(X_valid), torch.FloatTensor(y_valid))
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))

    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=config.batch_size)
    test_loader = DataLoader(test_dataset, batch_size=config.batch_size)

    return train_loader, valid_loader, test_loader, scalers, df



def train_model(model, train_loader, valid_loader, config):
    """训练模型"""
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, verbose=True
    )

    best_valid_loss = float('inf')
    patience = 10
    patience_counter = 0

    try:
        for epoch in range(config.epochs):
            model.train()
            train_losses = []
            train_accuracies = []

            # 添加进度条
            train_iterator = tqdm(train_loader,
                                desc=f'Epoch {epoch+1}/{config.epochs}',
                                leave=False)

            for batch_idx, (X, y) in enumerate(train_iterator):
                X, y = X.to(config.device), y.to(config.device)

                optimizer.zero_grad()
                output = model(X, y, config.teacher_forcing_ratio)
                loss, accuracy = custom_loss(output, y)

                loss.backward()
                optimizer.step()

                train_losses.append(loss.item())
                train_accuracies.append(accuracy.item())

            # 验证
            model.eval()
            valid_losses = []
            valid_accuracies = []

            with torch.no_grad():
                for X, y in valid_loader:
                    X, y = X.to(config.device), y.to(config.device)
                    output = model(X, None, 0.0)  # 验证时不使用teacher forcing
                    loss, accuracy = custom_loss(output, y)

                    valid_losses.append(loss.item())
                    valid_accuracies.append(accuracy.item())

            train_loss = np.mean(train_losses)
            valid_loss = np.mean(valid_losses)
            train_acc = np.mean(train_accuracies)
            valid_acc = np.mean(valid_accuracies)

            print(f'\nEpoch {epoch+1}/{config.epochs}')
            print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}')
            print(f'Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}')

            # 学习率调整
            scheduler.step(valid_loss)

            # 早停
            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                patience_counter = 0
                torch.save(model.state_dict(), 'best_model.pth')
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f'Early stopping after {epoch+1} epochs')
                    break

        # 加载最佳模型
        model.load_state_dict(torch.load('best_model.pth'))
        return model

    except Exception as e:
        print(f"Error during training: {str(e)}")
        raise


def prepare_test_data(df, config, test_size=0.1):
    """准备测试数据，使用残差预测"""
    # 选择特征
    features = ['Open', 'High', 'Low', 'Adj Close', 'Volume']
    target = 'Adj Close'

    # 计算目标变量的差分
    df['Adj Close_diff'] = df['Adj Close'].diff()
    df = df.dropna()  # 删除第一行（差分后的NaN）

    # 获取测试集
    test_start = int(len(df) * (1 - test_size))
    test_df = df[test_start:].copy()

    print(f"测试集大小: {len(test_df)} 条数据")
    print(f"测试集日期范围: {test_df['Date'].iloc[0]} 到 {test_df['Date'].iloc[-1]}")

    # 标准化
    scalers = {}
    test_scaled = pd.DataFrame()

    # 对原始特征进行标准化
    for column in features:
        if column == 'Volume':
            scalers[column] = StandardScaler()
        else:
            scalers[column] = MinMaxScaler()

        scalers[column].fit(df[column].values.reshape(-1, 1))
        test_scaled[column] = scalers[column].transform(
            test_df[column].values.reshape(-1, 1)
        ).flatten()

    # 对差分序列单独使用StandardScaler
    scalers['diff'] = StandardScaler()
    scalers['diff'].fit(df['Adj Close_diff'].values.reshape(-1, 1))
    test_scaled['Adj Close_diff'] = scalers['diff'].transform(
        test_df['Adj Close_diff'].values.reshape(-1, 1)
    ).flatten()

    # 创建序列
    X_test, y_test = [], []
    for i in range(len(test_scaled) - config.src_len - config.tgt_len + 1):
        # 输入序列（包含所有原始特征）
        X_test.append(test_scaled[features].iloc[i:i + config.src_len].values)
        # 目标序列（预测差分值）
        y_test.append(test_scaled['Adj Close_diff'].iloc[i + config.src_len:i + config.src_len + config.tgt_len].values)

    X_test = np.array(X_test)
    y_test = np.array(y_test).reshape(-1, config.tgt_len, 1)

    print(f"\n测试序列形状:")
    print(f"输入 (X_test): {X_test.shape}")
    print(f"目标 (y_test): {y_test.shape}")

    # 创建数据加载器
    test_dataset = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
    test_loader = DataLoader(test_dataset, batch_size=config.batch_size)

    return test_loader, scalers, test_df




def evaluate_model(model, test_loader, config, scalers, original_df):
    """评估模型性能并可视化预测结果（基于残差预测）"""
    model.eval()
    predictions_dict = {}

    # 获取测试集的日期
    test_size = len(test_loader.dataset)
    start_idx = len(original_df) - test_size - config.tgt_len
    test_dates = pd.to_datetime(original_df['Date'].values[start_idx:])

    try:
        with torch.no_grad():
            for batch_idx, (X, y) in enumerate(test_loader):
                X = X.to(config.device)
                y = y.to(config.device)

                output = model(X, None, 0.0)

                pred_numpy = output.cpu().numpy()
                actual_numpy = y.cpu().numpy()

                pred_flat = pred_numpy.reshape(-1, 1)
                actual_flat = actual_numpy.reshape(-1, 1)

                denorm_pred_diff = scalers['diff'].inverse_transform(pred_flat)
                denorm_actual_diff = scalers['diff'].inverse_transform(actual_flat)

                denorm_pred_diff = denorm_pred_diff.reshape(-1, config.tgt_len)
                denorm_actual_diff = denorm_actual_diff.reshape(-1, config.tgt_len)

                batch_size = X.size(0)
                for i in range(batch_size):
                    current_idx = batch_idx * config.batch_size + i
                    current_date = test_dates[current_idx]
                    base_price = original_df['Adj Close'].values[start_idx + current_idx]

                    pred_prices = np.zeros(config.tgt_len)
                    actual_prices = np.zeros(config.tgt_len)

                    # 计算第一步预测和实际价格
                    pred_prices[0] = base_price + denorm_pred_diff[i, 0]
                    actual_prices[0] = base_price + denorm_actual_diff[i, 0]

                    # 计算后续步骤的价格
                    for j in range(1, config.tgt_len):
                        pred_prices[j] = pred_prices[j-1] + denorm_pred_diff[i, j]
                        actual_prices[j] = actual_prices[j-1] + denorm_actual_diff[i, j]

                    target_dates = test_dates[current_idx + 1:current_idx + 1 + config.tgt_len]
                    if len(target_dates) == config.tgt_len:
                        predictions_dict[current_date] = {
                            'values': pred_prices,
                            'target_dates': target_dates,
                            'actuals': actual_prices,
                            'base_price': base_price
                        }

        # 转换为DataFrame格式
        results_df = []
        for pred_date, pred_info in predictions_dict.items():
            pred_values = pred_info['values']
            target_dates = pred_info['target_dates']
            actual_prices = pred_info['actuals']
            base_price = pred_info['base_price']

            # 计算第一步的趋势是否正确
            first_pred_trend = (pred_values[0] - base_price) > 0
            first_actual_trend = (actual_prices[0] - base_price) > 0
            first_trend_correct = first_pred_trend == first_actual_trend

            # 计算后续步骤的趋势
            pred_trends = np.diff(pred_values) > 0
            actual_trends = np.diff(actual_prices) > 0
            trend_correct = pred_trends == actual_trends

            # 添加第一步结果
            results_df.append({
                'prediction_date': pred_date,
                'target_date': target_dates[0],
                'prediction_step': 1,
                'predicted_value': pred_values[0],
                'actual_value': actual_prices[0],
                'trend_correct': first_trend_correct
            })

            # 添加后续步骤结果
            for j in range(1, len(pred_values)):
                results_df.append({
                    'prediction_date': pred_date,
                    'target_date': target_dates[j],
                    'prediction_step': j + 1,
                    'predicted_value': pred_values[j],
                    'actual_value': actual_prices[j],
                    'trend_correct': trend_correct[j-1]
                })

        results_df = pd.DataFrame(results_df)

        # 计算评估指标
        metrics_by_step = {}
        for step in range(1, config.tgt_len + 1):
            step_data = results_df[results_df['prediction_step'] == step]

            preds = step_data['predicted_value'].values
            actuals = step_data['actual_value'].values

            mse = np.mean((preds - actuals) ** 2)
            rmse = np.sqrt(mse)
            mae = np.mean(np.abs(preds - actuals))
            mape = np.mean(np.abs((actuals - preds) / actuals)) * 100
            acc = np.mean(step_data['trend_correct'].values) * 100

            metrics_by_step[step] = {
                'RMSE': rmse,
                'MAE': mae,
                'MAPE': mape,
                'ACC': acc,
                'samples': len(preds)
            }

        # 计算整体指标
        all_preds = results_df['predicted_value'].values
        all_actuals = results_df['actual_value'].values
        all_trends = results_df['trend_correct'].values

        # 计算整体趋势准确率（包括第一步）
        overall_acc = np.mean(all_trends) * 100

        overall_metrics = {
            'RMSE': np.sqrt(np.mean((all_preds - all_actuals) ** 2)),
            'MAE': np.mean(np.abs(all_preds - all_actuals)),
            'MAPE': np.mean(np.abs((all_actuals - all_preds) / all_actuals)) * 100,
            'ACC': overall_acc,
            'samples': len(all_preds)
        }

        # 保存结果
        results_df = results_df.sort_values(['prediction_date', 'prediction_step'])
        results_df.to_csv('prediction_results.csv', index=False, float_format='%.4f')
        print("\n预测结果已保存到 prediction_results.csv")

        # 可视化结果
        plot_prediction_results(results_df, metrics_by_step, overall_metrics)

        return metrics_by_step, overall_metrics, results_df

    except Exception as e:
        print(f"评估过程中出错: {str(e)}")
        raise

def calculate_trend_accuracy(y_true, y_pred):
    """计算趋势预测准确率
    y_true: 实际值序列
    y_pred: 预测值序列
    返回: 趋势预测准确率
    """
    # 计算实际变化方向 (1: 上涨, 0: 下跌或持平)
    actual_trend = (np.diff(y_true) > 0).astype(int)
    # 计算预测变化方向
    pred_trend = (np.diff(y_pred) > 0).astype(int)
    # 计算方向一致的比例
    accuracy = np.mean(actual_trend == pred_trend)
    return accuracy * 100  # 转换为百分比

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px


def plot_prediction_results(results_df, metrics_by_step, overall_metrics):
    """使用plotly绘制多步预测结果的详细可视化"""

    # 1. 创建多步预测分析图
    fig1 = go.Figure()

    # 绘制实际值
    actual_data = results_df.drop_duplicates('target_date')[['target_date', 'actual_value']].sort_values('target_date')
    fig1.add_trace(
        go.Scatter(
            x=actual_data['target_date'],
            y=actual_data['actual_value'],
            name='实际值',
            line=dict(color='blue', width=2),
            mode='lines'
        )
    )

    # 使用红色渐变色系
    colors = px.colors.sequential.Reds[2:]  # 跳过最浅的颜色

    # 对每个目标日期，收集并绘制所有可用的预测值
    all_target_dates = sorted(results_df['target_date'].unique())

    # 为每个预测步长绘制预测序列
    for step in range(1, lookahead + 1):
        # 获取特定步长的所有预测
        step_predictions = []
        for target_date in all_target_dates:
            # 找到对应的预测日期（向前推step天）
            pred_date = pd.to_datetime(target_date) - pd.Timedelta(days=step)
            pred_data = results_df[
                (results_df['prediction_date'] == pred_date) &
                (results_df['target_date'] == target_date)
            ]
            if not pred_data.empty:
                step_predictions.append({
                    'target_date': target_date,
                    'predicted_value': pred_data['predicted_value'].iloc[0]
                })

        if step_predictions:
            step_df = pd.DataFrame(step_predictions)
            # 计算颜色深度（步长越小，颜色越深）
            color_index = -(step)  # 使用负索引从最深的颜色开始

            fig1.add_trace(
                go.Scatter(
                    x=step_df['target_date'],
                    y=step_df['predicted_value'],
                    name=f't-{step}日预测',
                    line=dict(
                        color=colors[color_index],
                        width=1.5
                    ),
                    mode='lines',
                    opacity=1 - (step - 1) * 0.3 / lookahead  # 步长越大，透明度越高
                )
            )

    # 更新多步预测分析图的布局
    fig1.update_layout(
        height=600,
        title_text="多步预测分析",
        showlegend=True,
        hovermode='x unified',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        xaxis_title="日期",
        yaxis_title="指数"
    )

    fig1.show()

    # 2. 创建预测误差分析图
    fig2 = go.Figure()

    steps = sorted(metrics_by_step.keys())
    rmse_values = [metrics_by_step[s]['RMSE'] for s in steps]
    mae_values = [metrics_by_step[s]['MAE'] for s in steps]
    mape_values = [metrics_by_step[s]['MAPE'] for s in steps]
    acc_values = [metrics_by_step[s]['ACC'] for s in steps]

    # 添加误差指标柱状图
    fig2.add_trace(
        go.Bar(
            x=steps,
            y=rmse_values,
            name='RMSE',
            marker_color='blue',
            opacity=0.7
        )
    )

    fig2.add_trace(
        go.Bar(
            x=steps,
            y=mae_values,
            name='MAE',
            marker_color='red',
            opacity=0.7
        )
    )

    fig2.add_trace(
        go.Bar(
            x=steps,
            y=mape_values,
            name='MAPE(%)',
            marker_color='green',
            opacity=0.7
        )
    )

    # 添加趋势准确率曲线
    fig2.add_trace(
        go.Scatter(
            x=steps,
            y=acc_values,
            name='趋势准确率(%)',
            line=dict(color='purple', width=2),
            mode='lines+markers',
            yaxis='y2'
        )
    )

    # 更新预测误差分析图的布局
    fig2.update_layout(
        height=400,
        title_text="预测误差分析",
        showlegend=True,
        hovermode='x unified',
        xaxis_title="预测步长",
        yaxis_title="误差值",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ),
        yaxis2=dict(
            title="趋势准确率(%)",
            overlaying="y",
            side="right",
            range=[0, 100]
        )
    )

    fig2.show()

    # 3. 创建预测准确性散点图
    fig3 = make_subplots(
        rows=1, cols=len(steps),
        subplot_titles=[f'第{s}步预测' for s in steps]
    )

    for i, step in enumerate(steps, 1):
        step_data = results_df[results_df['prediction_step'] == step].dropna()

        fig3.add_trace(
            go.Scatter(
                x=step_data['actual_value'],
                y=step_data['predicted_value'],
                mode='markers',
                marker=dict(
                    size=5,
                    opacity=0.5
                ),
                name=f'第{step}步'
            ),
            row=1, col=i
        )

        # 添加对角线
        min_val = min(step_data['actual_value'].min(), step_data['predicted_value'].min())
        max_val = max(step_data['actual_value'].max(), step_data['predicted_value'].max())
        fig3.add_trace(
            go.Scatter(
                x=[min_val, max_val],
                y=[min_val, max_val],
                mode='lines',
                line=dict(color='red', dash='dash'),
                showlegend=False
            ),
            row=1, col=i
        )

    fig3.update_layout(
        height=400,
        title_text="预测准确性分析",
        showlegend=True
    )

    for i in range(len(steps)):
        fig3.update_xaxes(title_text="实际值", row=1, col=i+1)
        fig3.update_yaxes(title_text="预测值", row=1, col=i+1)

    fig3.show()



def main():
    try:
        # 创建配置实例
        config = Config()

        # 下载数据
        download_stock_data()


        # 读取数据
        print("加载数据...")
        df = pd.read_csv('000001.SS.csv')
        df = df.sort_values('Date')  # 确保按日期排序

        # 准备数据
        print("\n准备测试数据...")
        train_loader, valid_loader, test_loader, scalers, df = prepare_data(df, config)
        #test_loader, scalers, test_df = prepare_test_data(df, config, test_size=0.1)

        # 创建模型实例
        print("\n创建模型...")
        model = Galformer(config).to(config.device)


        # 训练模型
        print("\n开始训练...")
        model = train_model(model, train_loader, valid_loader, config)
        ## 加载预训练的模型权重
        #print("\n加载预训练模型权重...")
        #model.load_state_dict(torch.load('best_model.pth', map_location=config.device))

        # 准备测试数据
        print("\n准备测试数据...")
        test_loader, scalers, test_df = prepare_test_data(df, config, test_size=0.1)


        # 评估模型
        print("\n开始评估...")
        metrics_by_step, overall_metrics, results_df = evaluate_model(
            model,
            test_loader,
            config,
            scalers,
            df  # 传入完整的DataFrame以获取正确的日期
        )


        # 打印评估结果
        print("\n各预测步长的详细性能指标:")
        print("步长   | RMSE    | MAE     | MAPE(%) | ACC(%)  | 样本数")
        print("--------|---------|---------|---------|---------|--------")
        for step in sorted(metrics_by_step.keys()):
            m = metrics_by_step[step]
            acc_str = f"{m['ACC']:7.2f}" if m['ACC'] is not None else "   N/A  "
            print(f"第{step:2d}步  | {m['RMSE']:7.2f} | {m['MAE']:7.2f} | {m['MAPE']:7.2f} | {acc_str} | {m['samples']:7d}")

        print("\n整体预测性能:")
        print(f"RMSE: {overall_metrics['RMSE']:.2f}")
        print(f"MAE: {overall_metrics['MAE']:.2f}")
        print(f"MAPE: {overall_metrics['MAPE']:.2f}%")
        print(f"ACC: {overall_metrics['ACC']:.2f}%" if overall_metrics['ACC'] is not None else "ACC: N/A")
        print(f"总样本数: {overall_metrics['samples']}")

        # 结果已在evaluate_model中保存为CSV
        print("\n详细预测结果已保存到 prediction_results.csv")

    except Exception as e:
        print(f"发生错误: {str(e)}")
        raise
    finally:
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

if __name__ == "__main__":
    main()


##################################################
000001.SS
         Date         Open         High          Low        Close  \
0  2011-07-01  2767.833008  2778.667969  2752.966064  2759.362061   
1  2011-07-04  2770.939941  2813.270020  2770.939941  2812.818115   

     Adj Close  Volume  
0  2759.362061   92000  
1  2812.818115  122000  
##################################################
##################################################
AAPL
         Date       Open       High        Low      Close  Adj Close  \
0  2011-07-01  11.998214  12.267857  11.935714  12.259286  10.391948   
1  2011-07-05  12.250000  12.493929  12.232143  12.479643  10.578743   

      Volume  
0  435313200  
1  355054000  
##################################################
##################################################
BTC-USD
         Date        Open        High         Low       Close   Adj Close  \
0  2014-09-17  465.864014  468.174011  452.421997  457.334015  457.334015   
1  2014-09-18  456.8599

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(



Epoch 1/200
Train Loss: 49.1294, Train Acc: 0.5062
Valid Loss: 5.8937, Valid Acc: 0.4188



Epoch 2/200
Train Loss: 2.1519, Train Acc: 0.5153
Valid Loss: 0.7967, Valid Acc: 0.5531



Epoch 3/200
Train Loss: 1.2732, Train Acc: 0.5056
Valid Loss: 0.8670, Valid Acc: 0.4500


...


Epoch 17/200
Train Loss: 1.2499, Train Acc: 0.5106
Valid Loss: 0.7494, Valid Acc: 0.5437



Epoch 18/200
Train Loss: 1.2750, Train Acc: 0.5122
Valid Loss: 0.9431, Valid Acc: 0.5156



Epoch 19/200
Train Loss: 1.2347, Train Acc: 0.5070
Valid Loss: 0.8863, Valid Acc: 0.5000



Epoch 20/200
Train Loss: 1.2621, Train Acc: 0.4998
Valid Loss: 0.8082, Valid Acc: 0.4969
Early stopping after 20 epochs

准备测试数据...
测试集大小: 291 条数据
测试集日期范围: 2022-04-19 到 2023-06-29

测试序列形状:
输入 (X_test): (260, 30, 5)
目标 (y_test): (260, 2, 1)

开始评估...

预测结果已保存到 prediction_results.csv



各预测步长的详细性能指标:
步长   | RMSE    | MAE     | MAPE(%) | ACC(%)  | 样本数
--------|---------|---------|---------|---------|--------
第 1步  |   27.16 |   20.89 |    0.65 |   47.31 |     260
第 2步  |   38.01 |   29.71 |    0.93 |   46.92 |     260

整体预测性能:
RMSE: 33.03
MAE: 25.30
MAPE: 0.79%
ACC: 47.12%
总样本数: 520

详细预测结果已保存到 prediction_results.csv
